In [3]:
import pandas as pd
import numpy as np

# Seed for reproducibility
np.random.seed(123)

# Number of applicants
n = 1200

# Options for categorical data
genders = ["Male", "Female", "Other"]
ethnicities = ["Group_A", "Group_B", "Group_C"]
education_levels = ["High School", "Bachelors", "Masters", "PhD"]

# Create the synthetic recruitment dataset
data = pd.DataFrame({
    "Applicant_ID": range(1, n + 1),
    "Age": np.random.randint(20, 60, n),
    "Gender": np.random.choice(genders, n, p=[0.5, 0.45, 0.05]),
    "Ethnicity": np.random.choice(ethnicities, n, p=[0.4, 0.35, 0.25]),
    "Education": np.random.choice(education_levels, n, p=[0.3, 0.4, 0.2, 0.1]),
    "Years_Experience": np.random.randint(0, 20, n),
    "Skill_Score": np.random.randint(40, 100, n)  # Out of 100
})

# Shortlisting logic with bias patterns

# 1. Education + Skill Score bias
condition = (data["Education"].isin(["Masters", "PhD"])) & (data["Skill_Score"] > 70)
data.loc[condition, "Shortlisted"] = np.random.choice([1, 0], condition.sum(), p=[0.75, 0.25])
data.loc[~condition, "Shortlisted"] = np.random.choice([1, 0], (~condition).sum(), p=[0.35, 0.65])

# 2. Additional bias: Favor Ethnicity = Group_A
mask_group_a = data["Ethnicity"] == "Group_A"
data.loc[mask_group_a, "Shortlisted"] = np.random.choice([1, 0], mask_group_a.sum(), p=[0.55, 0.45])

# Preview the dataset
print(data.head())


   Applicant_ID  Age Gender Ethnicity  Education  Years_Experience  \
0             1   22   Male   Group_B  Bachelors                12   
1             2   48  Other   Group_B  Bachelors                12   
2             3   54   Male   Group_B  Bachelors                19   
3             4   58   Male   Group_C    Masters                 8   
4             5   37   Male   Group_C  Bachelors                16   

   Skill_Score  Shortlisted  
0           87          0.0  
1           89          0.0  
2           45          1.0  
3           41          0.0  
4           48          0.0  


In [4]:
# =========================================
# Task 1: Bias Detection in Development
# =========================================
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Prepare data
X = pd.get_dummies(data.drop(columns=["Applicant_ID", "Shortlisted"]), drop_first=True)
y = data["Shortlisted"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train baseline model
model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Baseline Accuracy:", accuracy_score(y_test, y_pred))

# Bias check by Gender and Ethnicity
df_test = X_test.copy()
df_test["y_true"] = y_test
df_test["y_pred"] = y_pred
df_test["Gender"] = data.loc[y_test.index, "Gender"]
df_test["Ethnicity"] = data.loc[y_test.index, "Ethnicity"]

bias_gender = df_test.groupby("Gender")["y_pred"].mean()
bias_ethnicity = df_test.groupby("Ethnicity")["y_pred"].mean()

print("\nPositive prediction rate by Gender:\n", bias_gender)
print("\nPositive prediction rate by Ethnicity:\n", bias_ethnicity)


Baseline Accuracy: 0.5055555555555555

Positive prediction rate by Gender:
 Gender
Female    0.325000
Male      0.326087
Other     0.437500
Name: y_pred, dtype: float64

Positive prediction rate by Ethnicity:
 Ethnicity
Group_A    0.447761
Group_B    0.216418
Group_C    0.326087
Name: y_pred, dtype: float64


In [5]:
# =========================================
# Task 2: Fairness Testing in Dev
# =========================================
def fairness_check(df, sensitive_attr, threshold=0.1):
    rates = df.groupby(sensitive_attr)["y_pred"].mean()
    disparity = rates.max() - rates.min()
    print(f"{sensitive_attr} Disparity: {disparity:.3f}")
    return disparity <= threshold

if not fairness_check(df_test, "Gender"):
    print("❌ Fairness check failed for Gender.")
else:
    print("✅ Fairness check passed for Gender.")

if not fairness_check(df_test, "Ethnicity"):
    print("❌ Fairness check failed for Ethnicity.")
else:
    print("✅ Fairness check passed for Ethnicity.")


Gender Disparity: 0.112
❌ Fairness check failed for Gender.
Ethnicity Disparity: 0.231
❌ Fairness check failed for Ethnicity.


In [6]:
# =========================================
# Task 3: Feature Role & Ethics Review
# =========================================
importances = pd.Series(model.feature_importances_, index=X.columns)
importances_sorted = importances.sort_values(ascending=False)
print(importances_sorted.head(10))

# Flag if socio-demographic features are high
socio_features = ["Gender_Male", "Gender_Other", "Ethnicity_Group_B", "Ethnicity_Group_C"]
for feat in socio_features:
    if feat in importances_sorted.head(10).index:
        print(f"⚠️ Feature '{feat}' is high in importance — consider mitigation.")


Skill_Score              0.305972
Age                      0.268212
Years_Experience         0.216665
Gender_Male              0.037853
Ethnicity_Group_B        0.035664
Education_High School    0.034966
Education_Masters        0.033892
Ethnicity_Group_C        0.028907
Education_PhD            0.022765
Gender_Other             0.015104
dtype: float64
⚠️ Feature 'Gender_Male' is high in importance — consider mitigation.
⚠️ Feature 'Gender_Other' is high in importance — consider mitigation.
⚠️ Feature 'Ethnicity_Group_B' is high in importance — consider mitigation.
⚠️ Feature 'Ethnicity_Group_C' is high in importance — consider mitigation.


In [7]:
# =========================================
# Task 4: Ethics-Aware Agile User Stories
# =========================================
user_stories = [
    "As a recruiter, I want the model to avoid bias based on gender so that all applicants are evaluated fairly.",
    "As an HR manager, I want transparency in model decisions so that candidates can be given clear feedback."
]

for story in user_stories:
    print("-", story)


- As a recruiter, I want the model to avoid bias based on gender so that all applicants are evaluated fairly.
- As an HR manager, I want transparency in model decisions so that candidates can be given clear feedback.


In [8]:
# =========================================
# Task 5: Logging & Accountability
# =========================================
import datetime

MODEL_VERSION = "v1.0"
prediction_log = []

def predict_and_log(applicant_data):
    pred = model.predict(applicant_data)[0]
    log_entry = {
        "timestamp": datetime.datetime.now().isoformat(),
        "input": applicant_data.to_dict(orient='records')[0],
        "prediction": int(pred),
        "model_version": MODEL_VERSION
    }
    prediction_log.append(log_entry)
    return pred

# Example prediction
sample_applicant = X_test.iloc[[0]]
predict_and_log(sample_applicant)
prediction_log[:2]


[{'timestamp': '2025-08-13T10:39:09.320615',
  'input': {'Age': 44,
   'Years_Experience': 1,
   'Skill_Score': 44,
   'Gender_Male': False,
   'Gender_Other': False,
   'Ethnicity_Group_B': False,
   'Ethnicity_Group_C': False,
   'Education_High School': True,
   'Education_Masters': False,
   'Education_PhD': False},
  'prediction': 0,
  'model_version': 'v1.0'}]

In [9]:
# =========================================
# Task 6: Real-Time Monitoring
# =========================================
dp_gender = bias_gender.max() - bias_gender.min()
dp_ethnicity = bias_ethnicity.max() - bias_ethnicity.min()

if dp_gender > 0.1:
    print("⚠️ Alert: Gender fairness threshold exceeded.")
else:
    print("✅ Gender fairness acceptable.")

if dp_ethnicity > 0.1:
    print("⚠️ Alert: Ethnicity fairness threshold exceeded.")
else:
    print("✅ Ethnicity fairness acceptable.")


⚠️ Alert: Gender fairness threshold exceeded.
⚠️ Alert: Ethnicity fairness threshold exceeded.


In [10]:
# =========================================
# Task 7: Privacy Controls
# =========================================
import hashlib

data["Applicant_ID"] = data["Applicant_ID"].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
data.head()


,Applicant_ID,Age,Gender,Ethnicity,Education,Years_Experience,Skill_Score,Shortlisted
0,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,22,Male,Group_B,Bachelors,12,87,0.0
1,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,48,Other,Group_B,Bachelors,12,89,0.0
2,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,54,Male,Group_B,Bachelors,19,45,1.0
3,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,58,Male,Group_C,Masters,8,41,0.0
4,ef2d127de37b942baad06145e54b0c619a1f22327b2ebb...,37,Male,Group_C,Bachelors,16,48,0.0


In [11]:
# =========================================
# Task 8: Inclusivity Feedback Loop
# =========================================
feedback = pd.DataFrame({
    "Applicant_ID": ["new_1", "new_2"],
    "Age": [29, 35],
    "Gender": ["Other", "Other"],
    "Ethnicity": ["Group_C", "Group_C"],
    "Education": ["Bachelors", "Masters"],
    "Years_Experience": [3, 5],
    "Skill_Score": [72, 80],
    "Shortlisted": [1, 1]
})

data_updated = pd.concat([data, feedback], ignore_index=True)
print("Updated dataset size:", data_updated.shape)


Updated dataset size: (1202, 8)


In [12]:
# =========================================
# Task 9: Ethical Incident Simulation
# =========================================
incident_report = {
    "Detection": "User complaint about potential bias against Ethnicity Group_C.",
    "Investigation": "Analyze model predictions to confirm disparity.",
    "Communication": "Notify compliance and HR departments; prepare external statement.",
    "Remediation": "Retrain model with balanced dataset; remove Ethnicity feature."
}

for step, detail in incident_report.items():
    print(f"{step}: {detail}")


Detection: User complaint about potential bias against Ethnicity Group_C.
Investigation: Analyze model predictions to confirm disparity.
Communication: Notify compliance and HR departments; prepare external statement.
Remediation: Retrain model with balanced dataset; remove Ethnicity feature.
